In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import h5py
import os
import sys
import scipy
import damselfly as df
import scipy.signal
import scipy.stats

PATH = '/storage/home/adz6/group/project'
RESULTPATH = os.path.join(PATH, 'results/damselfly')
PLOTPATH = os.path.join(PATH, 'plots/damselfly')
DATAPATH = os.path.join(PATH, 'damselfly/data/datasets')
SIMDATAPATH = os.path.join(PATH, 'damselfly/data/sim_data')

"""
Date: 6/25/2021
Description: template
"""


def MakeTemplates(signals, var =  1.38e-23 * 10 * 50 * 200e6):
    norm_mat = 1 / np.sqrt(var * np.diag(np.matmul(signals, signals.conjugate().T)))

    templates = norm_mat.reshape((*norm_mat.shape, 1)).repeat(signals.shape[-1], axis=-1) * signals
    
    return templates


In [ ]:
os.listdir(os.path.join(PATH, 'datasets/data/dl'))

In [ ]:
data = os.path.join(PATH, 'datasets/data/dl', '211215_dl_classification_84_25_2cm_slice1_sample1x8192.h5')
h5datafile = h5py.File(data, 'r')

In [ ]:
h5datafile['train']['data'].shape

In [ ]:
pitch_angles = h5datafile['train']['meta']['theta_min'][:]
energies = h5datafile['train']['meta']['energy'][:]

# manipulate data

In [ ]:
real = h5datafile['train']['data'][:, 0, :]
imag = h5datafile['train']['data'][:, 1, :]

cmplx_data = real + 1j * imag

# whole dataset PCA

In [ ]:
data_matrix = np.matmul(cmplx_data.T, cmplx_data) # (nsample x nsample)

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako_r', as_cmap=True)

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(np.log(abs(data_matrix)), interpolation='none', cmap=cmap)
fig.colorbar(img)


In [ ]:
evals, evecs = np.linalg.eig(data_matrix)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
cmap = sns.color_palette('mako_r', as_cmap=True)

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(evals))

#ax.set_xlim(0, 2048)

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
cmap = sns.color_palette('mako_r', as_cmap=True)

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(evecs.T[0, :]))


print(np.argwhere(abs(evecs.T[0, :] >= 0.02)).shape)

#ax.plot(evecs.T[, :])
#ax.set_xlim(0, 1024)

In [ ]:
os.listdir(os.path.join(PATH, 'results', 'damselfly','pca', ''))

In [ ]:
os.mkdir(os.path.join(PATH, 'results', 'damselfly','pca'))

In [ ]:
# save result


np.savez(os.path.join(PATH, 'results', 'damselfly','pca', '211208_84_25_2cm_pca'), evals=evals, evecs=evecs)

# use meta data to select a range of pitch angles

In [ ]:
theta_min = 84
theta_max = 90

energy = 18575

indices = np.logical_and(pitch_angles >= theta_min, pitch_angles <= theta_max)

indices = np.logical_and(indices, energies == energy)

data_matrix = np.matmul(cmplx_data[indices, :].T, cmplx_data[indices]) # (nsample x nsample)

In [ ]:
indices.sum()

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('mako_r', as_cmap=True)

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

img = ax.imshow(np.log(abs(data_matrix)), interpolation='none', cmap=cmap)
fig.colorbar(img)


In [ ]:
torch.cuda.is_available()

In [ ]:
eigout = torch.linalg.eig(torch.as_tensor(data_matrix).cuda())

In [ ]:
evecs = eigout[1].cpu().numpy()
evals = eigout[0].cpu().numpy()

In [ ]:
plt.plot(abs(evecs[:, 0]))

In [ ]:
sns.set_theme(context='talk', style='whitegrid')
cmap = sns.color_palette('mako_r', as_cmap=True)

fig = plt.figure(figsize=(13,8))
ax = fig.add_subplot(1,1,1)

ax.plot(abs(evals))

#ax.set_xlim(0, 2048)

In [ ]:
np.savez(os.path.join(PATH, 'results', 'damselfly','pca', '211215_88-89_18575_2cm_pca'), evals=evals, evecs=evecs, matrix = data_matrix)